In [1]:
import torch as th 
from torch_geometric.data import Data 
from torch_geometric.loader import DataLoader
from model import Generator, Discriminator, gradient_penalty

In [ ]:
""" 
Import Data
"""


In [ ]:
""" 
Hyperparameters
"""

# Optimizer params
g_lr = 0.001 
d_lr = 0.001
b1 = 0.5 
b2 = 0.999  

# WGAN params
N_critic = 5            # nr of times to train discriminator more
lambda_gp = 10          # gradient penalty hyperpraram

# Training params
MAX_EPOCHS = 500

In [ ]:
""" 
Model Definitions
"""
generator = Generator()
discriminator = Discriminator()


# Optimizers
optimizer_G = th.optim.Adam(generator.parameters(), lr=g_lr, betas=(b1, b2)) 
optimizer_D = th.optim.Adam(discriminator.parameters(), lr=d_lr, betas=(b1, b2))

In [ ]:
# Main training loop
def train(generator, discriminator, optimizer_g, optimizer_d, data_loader):
    for epoch in range(MAX_EPOCHS):
        for data in data_loader:
            # Data: Torch geometric Data(x=[node_features], edge_index=[])

            # Create noise vector
            noise = th.randn(data.x.shape)

            # Create new data object with noise and same edge_index
            # TODO: How to batch noise vectors alongside data?
            noise_data = Data(x=noise, edge_index = data.edge_index)

            # Input noise_data into generator
            fake = Generator(noise_data)

            # Determine discriminator score
            fake_score = Discriminator(fake)
            real_score = Discriminator(data)

            # Compute gradient penalty

            # Compute losses
            # TODO: WGAN-GP loss
            # TODO: Train discriminator 5x more
